In [1]:
import numpy as np
import tensorflow as tf

import argparse
import time
import os


# from LSTM import Model
# from LSTM import load_pretrained_model
import LSTM
from LSTM import *
from importlib import reload
reload(LSTM)
from dataloader import DataProcess
from sample import *

In [ ]:
def init_args():
    
        args = {}
        args['rnn_size'] = 100 
        args['tsteps'] = 256 
        args['batch_size'] = 32
        args['num_batches'] = 500 #32 
        args['num_mixtures'] = 8 
        args['alphabet'] = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
        args['tsteps_per_ascii'] = 25
        args['epochs'] = 250 #100
        args['path'] = 'saved2/model.ckpt'
        args['biases'] = 1.0
        args['logs_dir'] = './logs/'
        args['save_path'] = 'saved2/model3/model.ckpt' #'saved2/model.ckpt-3000'
        args['load_path'] = 'saved2/model3/model.ckpt-10100'
        args['sleep_time'] = 60*5
        args['grad_clip'] = 10
        args['n_to_save'] = 500
        args['scale_factor'] = 50
        args['gap'] = 500
        return args

def load_pretrained_model(model, path):
        global_step = 0
        load_was_success = True
        try:
            save_dir = '/'.join(path.split('/')[:-1])
            ckpt = tf.train.get_checkpoint_state(save_dir)
            load_path = ckpt.model_checkpoint_path
            model.saver.restore(model.sess, load_path)
            #load_was_success = True
        except Exception as e:
            #print(e)
            load_was_success = False
        else:
            model.saver = tf.train.Saver(tf.global_variables())
            global_step = int(load_path.split('-')[-1])
            load_was_success = True
        return load_was_success, global_step
    
def train_model():
    sess = tf.Session()
    args = init_args()
    data_loader = DataProcess(args)
    model = Model(args)
    load_was_success, global_step = load_pretrained_model(model, args['save_path'])
    v_x, v_y = data_loader.get_validation_data()
    valid_inputs = {model.input: v_x, model.output: v_y}
    plot_loss = []
    
    for e in range(int(global_step/args['num_batches']), args['epochs']):

        c0, c1, c2 = model.istate_cell0.c.eval(), model.istate_cell1.c.eval(), model.istate_cell2.c.eval()
        h0, h1, h2 = model.istate_cell0.h.eval(), model.istate_cell1.h.eval(), model.istate_cell2.h.eval()

        for b in range(global_step%args['num_batches'], args['num_batches']):
#             print(b)

            i = e * args['num_batches'] + b
            if global_step is not 0 : i+=1 ; global_step = 0

            if i % args['n_to_save'] == 0 and (i > 0):
                model.saver.save(model.sess, args['save_path'], global_step = i) ;

            start = time.time()
            x, y = data_loader.get_next_batch()
#             print("printing x and y")
#             print(len(x))
#             print(len(x[0]))
#             print(len(x[1]))
#             print(len(y))
#             print(len(y[0]))
#             print(len(y[1]))
#             print("done printing x and y")

            feed = {model.input: x, model.output: y, \
                    model.istate_cell0.c: c0, model.istate_cell1.c: c1, model.istate_cell2.c: c2, \
                    model.istate_cell0.h: h0, model.istate_cell1.h: h1, model.istate_cell2.h: h2}
#             print("printing loss")
            #print(model.sess.run([model.cost, model.train_op], feed))
            [train_loss, _] = model.sess.run([model.cost, model.train_op], feed)
#             feed.update(valid_inputs)
#             [valid_loss] = model.sess.run([model.cost], feed)
            
#             running_average = running_average*remember_rate + train_loss*(1-remember_rate)

            end = time.time()
            
            if i % 10 is 0:
                print("train_loss: " + str(i))
                print(train_loss)
                plot_loss.append(train_loss)

def sample_model(logger=None):
    args = init_args()
    args['tsteps'] = 1
    args['batch_size'] = 1
#     if args['text'] == '':
    strings = ['Machine generated handwriting'] # test strings
#     else:
#         strings = [args['text']]

    model = Model(args)

    load_was_success, global_step = load_pretrained_model(model, args['load_path'])

    if load_was_success:
        for s in strings:
            print(s)
            strokes = sample(s, model, args)
            print(strokes)

            w_save_path = '{}figures/iter-{}-w-{}'.format(args['logs_dir'], global_step, s[:10].replace(' ', '_'))
            g_save_path = '{}figures/iter-{}-g-{}'.format(args['logs_dir'], global_step, s[:10].replace(' ', '_'))
            l_save_path = '{}figures/iter-{}-l-{}'.format(args['logs_dir'], global_step, s[:10].replace(' ', '_'))

            gauss_plot(strokes, 'Heatmap for "{}"'.format(s), figsize = (2*len(s),4), save_path="./gauss2.png")
            line_plot(strokes, 'Line plot for "{}"'.format(s), figsize = (len(s),2), save_path="./line2.png")
            print("plotted")

    else:
        print("load failed, sampling canceled")

    if True:
        tf.reset_default_graph()
        time.sleep(args['sleep_time'])
        sample_model(logger=logger)

            
            
            
            
            
            
            
            
            
    

In [ ]:
sample_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500


/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Machine generated handwriting
[[-0.06304365  0.05827764  0.39961857  0.37517318  0.04218041  1.        ]
 [-0.10822599 -0.07449429  0.39198259  0.3644526   0.06254855  1.        ]
 [-0.15468834 -0.2086795   0.39222276  0.36438072  0.0648115   1.        ]]


/Users/saumyap/writing-synthesis/sample.py:14: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  x = np.random.multivariate_normal(mean, cov, 1)


plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[-0.0836852   0.04639425  0.33601421  0.39892101  0.10179562  1.        ]
 [-0.14632735  0.10454838  0.39940882  0.37552246  0.04236084  1.        ]
 [-0.20789233  0.16223431  0.39915472  0.37595335  0.04302733  1.        ]]
plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[-0.04456469 -0.13183354  0.39168584  0.36447984  0.06126065  1.        ]
 [ 0.00268633 -0.21434621  0.33648431  0.39398271  0.02925966  1.        ]
 [ 0.06090712 -0.23025257  0.40643376  0.36813509  0.06069393  1.        ]]
plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[-0.04456469 -0.13183354  0.39168584  0.36447984  0.06126065  1.        ]
 [-0.04868716 -0.02155548  0.35116673  0.33460835 -0.14545113  1.        ]
 [-0.09525543 -0.1557196   0.39225802  0.36447269  0.06487728 

/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[-0.11938497  0.09983824  0.36765245  0.3535938  -0.01648015  1.        ]
 [-0.12336163  0.21018514  0.35113004  0.33468297 -0.1456411   1.        ]
 [-0.18465153  0.26790965  0.3990736   0.37596244  0.04304182  1.        ]]
plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[ 0.05935441 -0.01413636  0.40644163  0.36687469  0.0597727   1.        ]
 [ 0.01407485 -0.14666173  0.39185423  0.36453867  0.06248274  1.        ]
 [-0.04870205 -0.08865126  0.39935377  0.37627807  0.04347752  1.        ]]
plotted
INFO:tensorflow:Restoring parameters from saved2/model3/model.ckpt-102500
Machine generated handwriting
[[ 0.05935441 -0.01413636  0.40644163  0.36687469  0.0597727   1.        ]
 [-0.02414563  0.03223248  0.33634597  0.39908764  0.1015293   1.        ]
 [-0.13926439  0.13375106  0.36691263  0.35298538 -0.01549119 